# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Glioblastoma'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = False

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [79]:
trait_subdir = "TCGA_Glioblastoma_(GBM)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.GBM.sampleMap_GBM_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.GBM.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['CDE_DxAge',
 'CDE_alk_chemoradiation_standard',
 'CDE_chemo_adjuvant_alk',
 'CDE_chemo_adjuvant_tmz',
 'CDE_chemo_alk',
 'CDE_chemo_alk_days',
 'CDE_chemo_alk_long',
 'CDE_chemo_tmz',
 'CDE_chemo_tmz_days',
 'CDE_chemo_tmz_long']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['CDE_DxAge', 'CDE_alk_chemoradiation_standard', 'CDE_chemo_adjuvant_alk', 'CDE_chemo_adjuvant_tmz', 'CDE_chemo_alk', 'CDE_chemo_alk_days', 'CDE_chemo_alk_long', 'CDE_chemo_tmz', 'CDE_chemo_tmz_days', 'CDE_chemo_tmz_long', 'CDE_missing', 'CDE_missingflag', 'CDE_previously_treated', 'CDE_radiation_adjuvant', 'CDE_radiation_adjuvant_standard', 'CDE_radiation_adjuvant_standard_probable', 'CDE_radiation_any', 'CDE_radiation_standard', 'CDE_radiation_standard_probable', 'CDE_sou

In [11]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [12]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [44.0, nan, 50.0, 59.0, 56.0],
 'days_to_birth': [-16179.0, nan, -18341.0, -21617.0, -20516.0]}

In [13]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [14]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', nan, 'MALE', 'MALE', 'FEMALE']}

In [15]:
gender_col = 'gender'

In [16]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [17]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [18]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Glioblastoma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,1,78.0,1.0,1.021008,1.647774,-0.531035,-0.351872,0.494722,1.29429,0.729706,...,-0.748878,3.893183,1.209067,-1.28139,-0.373777,-0.333686,-0.108127,-1.109795,-4.368933,-0.521512
TCGA-02-0055-01,1,62.0,0.0,-0.864792,1.503874,-0.531035,-0.344872,0.832622,0.59429,-0.377494,...,0.674422,-0.945017,1.316667,-1.28139,0.018223,0.892514,-2.114027,-0.042395,-0.666033,0.774588
TCGA-02-2483-01,1,43.0,1.0,0.009808,-0.601726,-0.531035,-0.119072,0.968922,0.70029,0.957806,...,-0.748878,-1.079017,0.871767,-1.28139,0.140823,-1.308686,0.805673,-1.449395,-4.345033,-0.006812
TCGA-02-2485-01,1,53.0,1.0,0.959008,4.193574,-0.531035,-0.371972,0.509622,0.57919,0.987006,...,-0.748878,-0.621917,0.544567,-1.28139,0.216023,-0.576286,1.149873,-0.168395,-4.245433,-0.209912
TCGA-02-2486-01,1,64.0,1.0,0.412908,-0.923626,-0.531035,0.142528,0.719722,1.56289,0.418406,...,2.594022,-0.900217,2.752067,-1.28139,0.424823,1.443614,-2.604227,0.578805,-4.375333,0.508888


In [19]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 166 samples.


In [20]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Glioblastoma', the least common label is '1' with 166 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 60.0
  75%: 69.0
Min: 21.0
Max: 89.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 59 occurrences. This represents 35.54% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [21]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [22]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Glioblastoma/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [23]:
dataset = 'GEO'
trait_subdir = "Glioblastoma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE39144', 'GSE226976', 'GSE249289', 'GSE231994']

Repeat the below steps for all the accession numbers

In [66]:
cohort = accession_num = "GSE231994"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Glioblastoma/GSE231994/GSE231994_family.soft.gz',
 '../../DATA/GEO/Glioblastoma/GSE231994/GSE231994_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [67]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Cancer-immune expressional variation in glioblastoma second surgery samples to compare progression and pseudoprogression"
!Series_summary	"We performed cancer-immune gene expression analysis on a case series of glioblastoma second surgery samples due to novel enhancement following chemoradiation that were confirmed based on clinico-pathologic outcome as disease progression (PD) or pseudoprogression (psPD). This was accomplished using an nCounter Pancancer 360 IO panel. Our goals were to (1) determine if psPD events could be distinguished from PD events using differences in immune cell activation versus cancer cell proliferation and (2) examine whether samples stratified based on their molecular profile in the same way as documented clinico-pathologic diagnosis."
!Series_overall_design	"NanoString nCounter pancancer 360 IO panel testing was performed on 48 glioblastoma samples with 27 showing progression (PD) and 21 showing pseudoprogression (psPD) taken from brain resect

In [68]:
clinical_data.head()

,!Sample_geo_accession,GSM7307106,GSM7307107,GSM7307108,GSM7307109,GSM7307110,GSM7307111,GSM7307112,GSM7307113,GSM7307114,...,GSM7307144,GSM7307145,GSM7307146,GSM7307147,GSM7307148,GSM7307149,GSM7307150,GSM7307151,GSM7307152,GSM7307153
0,!Sample_characteristics_ch1,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,...,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma,diagnosis: glioblastoma
1,!Sample_characteristics_ch1,disease state: PD,disease state: PD,disease state: PD,disease state: PD,disease state: PD,disease state: PD,disease state: PD,disease state: PD,disease state: PD,...,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD,disease state: psPD
2,!Sample_characteristics_ch1,slide tissue area: 297.326,slide tissue area: 301.486,slide tissue area: 214.275,slide tissue area: 227.373,slide tissue area: 208.533,slide tissue area: 207.734,slide tissue area: 171.682,slide tissue area: 199.439,slide tissue area: 191.703,...,slide tissue area: 190.942,slide tissue area: 162.545,slide tissue area: 111.032,slide tissue area: 82.399,slide tissue area: 45.319,slide tissue area: 307.083,slide tissue area: 100.3,slide tissue area: 97.35,slide tissue area: 87.524,slide tissue area: 25.328
3,!Sample_characteristics_ch1,side: Right,side: Left,side: Right,side: Right,side: Left,side: Left,side: Left,side: Left,side: Left,...,side: Left,side: Left,side: Left,side: Right,side: Left,side: Left,side: Left,side: Left,side: Right,side: Right
4,!Sample_characteristics_ch1,lobe: Temporal,lobe: Frontal,lobe: Parietal,lobe: Parietal,lobe: Frontal,lobe: Frontal,lobe: Frontal,lobe: Temporal,lobe: Frontal,...,lobe: Frontal,lobe: Parietal,lobe: Frontal,lobe: Parietal,lobe: Temporal,lobe: Parietal,lobe: Occipital,lobe: Occipital,lobe: Temporal,lobe: Frontal


In [69]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['diagnosis: glioblastoma'],
 1: ['disease state: PD', 'disease state: psPD'],
 2: ['slide tissue area: 297.326',
  'slide tissue area: 301.486',
  'slide tissue area: 214.275',
  'slide tissue area: 227.373',
  'slide tissue area: 208.533',
  'slide tissue area: 207.734',
  'slide tissue area: 171.682',
  'slide tissue area: 199.439',
  'slide tissue area: 191.703',
  'slide tissue area: 178.5971',
  'slide tissue area: 176.424',
  'slide tissue area: 175.009',
  'slide tissue area: 170.739',
  'slide tissue area: 168.653',
  'slide tissue area: 157.995',
  'slide tissue area: 171.608',
  'slide tissue area: 149.764',
  'slide tissue area: 148.689',
  'slide tissue area: 144.718',
  'slide tissue area: 134.946',
  'slide tissue area: 110.68',
  'slide tissue area: 105.215',
  'slide tissue area: 102.329',
  'slide tissue area: 94.864',
  'slide tissue area: 80.072',
  'slide tissue area: 69.827',
  'slide tissue area: 49.013',
  'slide tissue area: 439.904',
  'slide tissue area: 

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [70]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Glioblastoma\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Glioblastoma\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [74]:
# is_gene_availabe = True
is_gene_availabe = False
trait_row_id = 0
trait_type = 'binary'

age_row_id = None
# age_row_id = 2

gender_row_id = None
# gender_row_id = 1

In [75]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [73]:
# def convert_trait(value):
#     if value == 'sample type: recurrent glioma':
#         return 1
#     else:
#         return None

def convert_trait(value):
    return 1

def convert_age(value):
    try:
        # Split the string and extract the age part
        age_str = value.split(': ')[-1]
        return int(age_str)  # Convert the extracted age to an integer
    except (ValueError, TypeError, IndexError):
        return None  # Unable to convert or unknown age
    
def convert_gender(value):
    if value == 'Sex: Male':
        return 1
    elif value == 'Sex: Female':
        return 0
    else:
        return None

In [62]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM7933102,GSM7933103,GSM7933104,GSM7933105,GSM7933106,GSM7933107,GSM7933108,GSM7933109,GSM7933110,GSM7933111,...,GSM7933139,GSM7933140,GSM7933141,GSM7933142,GSM7933143,GSM7933144,GSM7933145,GSM7933146,GSM7933147,GSM7933148
Glioblastoma,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Age,61,61,61,61,61,61,61,61,61,61,...,67,67,67,67,67,61,61,61,61,61
Gender,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [41]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM7089240,GSM7089241,GSM7089242,GSM7089243,GSM7089244,GSM7089245,GSM7089246,GSM7089247,GSM7089248,...,GSM7089278,GSM7089279,GSM7089280,GSM7089281,GSM7089282,GSM7089283,GSM7089284,GSM7089285,GSM7089286,GSM7089287
0,A2M,19.996867,17.782077,19.248811,16.879339,19.118102,17.502348,18.812564,18.621958,18.589620,...,18.410078,18.794772,17.912146,17.629865,16.658951,19.288412,17.345720,17.631606,18.809379,19.337605
1,ABCF1,7.020822,5.888663,6.305956,5.550797,6.375912,6.378189,5.832269,5.824305,5.723866,...,5.848054,6.361730,6.454851,7.526675,5.848691,5.861923,6.034203,6.207368,6.559762,5.492295
2,ACVR1C,9.206051,10.816495,8.825713,10.624323,10.128836,10.444774,9.761350,10.427556,10.515902,...,9.086820,7.922423,6.965619,8.797730,9.825390,8.867010,11.905799,9.330081,10.930353,9.858440
3,ADAM12,12.109527,10.732307,15.535675,8.049131,11.790075,11.389109,12.303945,10.563954,9.575288,...,12.329854,12.313015,12.966596,9.620433,12.428526,11.303405,9.228345,12.770426,12.999017,12.886462
4,ADGRE1,10.540421,9.478096,10.346792,7.918908,11.456320,9.159351,11.596414,10.846055,9.445066,...,11.891510,12.934114,9.536329,8.835707,8.693442,10.014485,9.960620,10.458000,11.283832,10.993279


In [42]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['A2M',
 'ABCF1',
 'ACVR1C',
 'ADAM12',
 'ADGRE1',
 'ADM',
 'ADORA2A',
 'AKT1',
 'ALDOA',
 'ALDOC',
 'ANGPT1',
 'ANGPT2',
 'ANGPTL4',
 'ANLN',
 'APC',
 'APH1B',
 'API5',
 'APLNR',
 'APOE',
 'APOL6']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [43]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B', 'API5', 'APLNR', 'APOE', 'APOL6']\n"

If not required, jump directly to the gene normalization step

In [44]:
requires_gene_mapping = False

In [46]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [143]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

'\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_id_key = key_name1\ngene_symb_key = key_name2\n\nGene annotation dictionary:\n{\'ID\': [\'1007_s_at\', \'1053_at\', \'117_at\', \'121_at\', \'1255_g_at\'], \'GB_ACC\': [\'U48705\', \'M87338\', \'X51757\', \'X69699\', \'L36861\'], \'SPOT_ID\': [nan, nan, nan, nan, nan], \'Species Scientific Name\': [\'Homo sapiens\', \'Homo sapiens\', \'Homo sapiens\', \'Homo sapiens\', \'Homo sapiens\'], \'Annotation Date\': [\'Oct 6, 2014\', \'Oct 6, 2014\', \'Oct 6, 2014\', \'Oct 6, 2014\', \'Oct 6, 2014\'], \'Sequence Type\': [\'Exemplar sequence\', \'Exemplar sequence\', \'Exemplar sequence\', \'Exemplar sequence\', \'Exemplar sequence\'], \'Sequence Source\': [\'Affymetrix Proprietary Da

In [144]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [47]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [48]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [49]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 48 samples.


In [50]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Glioblastoma', the least common label is '1.0' with 48 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.



True

In [51]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [52]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


End: there is no usable data.

In [5]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [155]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [195]:
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
0,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
1,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
2,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
3,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
4,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [198]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [199]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [200]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [201]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [202]:
cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 83.16% ± 5.64%


In [203]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [204]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0112
Found 100 significant genes associated with the trait 'Thyroid Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
     GABRB2     0.199853       9.445009e-24
       LRP4     0.153916       3.454747e-11
    SHROOM4     0.150237       3.783756e-09
      HMGA2     0.122833       4.533386e-08
       EPS8     0.148314       3.922968e-07
    KLHDC8A     0.103442       3.996388e-07
       LIPH     0.126974       5.203673e-07
      MPZL2     0.123835       7.878797e-07
      GABRD     0.085857       3.424810e-06
       GJB3     0.104297       7.705250e-06
   C6orf168     0.106365       1.189886e-05
        FN1     0.142549       1.760688e-05
      KCNQ3     0.132879       2.031323e-05
      GRHL3     0.103693       2.317901e-05
    ZCCHC12     0.091659       3.351467e-05
     ACTBL2     0.094812       6.476432e-05
      SYT12     0.127073       8.24

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
